# Coding Your Own Linear Regression Model

One task that you will almost certainly be required to do other data science courses (especially if you are a MIDS student) is to write up some of your statistical / machine learning models from scratch. This can be a very valuable exercise, as it ensures that you understand what is actually going on behind the scenes of the models you use ever day, and that you don't just think of them as "black boxes". 

To get a little practice doing this, today you will be coding up your own linear regression model! 

(If you are using this site but aren't actually in this class, you are welcome to skip this exercise if you'd like -- this is more about practicing Python in anticipation of the requirements of other courses than developing your applied data science skills.) 

There are, broadly speaking, two approaches you can take to coding up your own model: 

1. you can write the model by defining a new function, or 
2. you can write the model by defining a new class with associated methods (making a model that works the way a model works in `scikit-learn`). 

Whether you do 1 or 2 is very much a matter of choice and style. Approach one, for example, is more consistent with what is called a *functional* style of programming, while approach two is more consistent with an *object-oriented* style of programming. Python can readily support both approaches, so either would work fine. 

In these exercises, however, I will ask you to use approach number 2 as this *tends* to be the more difficult approach, and so practicing approach 2 will be extra useful in preparing you for other classes (HA! Pun...). In particular, our goal is to implement a linear regression model that has the same "initialize-fit-predict-score" API (application programming interface -- a fancy name for the methods a class supports) as `scikit-learn` models. That means your model should be able to do all of the following:

1. **Initialize** a new model. 
2. **Fit** a linear model when given a numpy vector (`y`) and a numpy matrix (`X`) with the syntax `my_model.fit(X, y)`. 
3. **Predict** values when given a new `numpy` matrix (`X_test`) with the syntax `my_model.predict(X_test)`. 
4. Return the **model coefficients** through the property `my_model.coefficients` (not quite what is used in `sklearn`, but let's use that interface). 

Also, bear in mind that throughout these exercises, we'll be working in `numpy` instead of `pandas`, just as we do in `scikit-learn`. So assume that before using your model, your user has already converted their data from `pandas` into `numpy` arrays. 

In [1]:
import pandas as pd

pd.set_option("mode.copy_on_write", True)
results = {}

**(1)** Define a new Class called `MyLinearModel` with methods for `__init__`, `fit`, `predict`, and an attribute for `coefficients`. For now, we don't need any initialization *arguments*, just an `__init__` function. 

As you get your code outline going, start by just having each method `pass`:

```python
def my_method(self):
    pass
```

This will allow your methods to run without errors (they just don't do anything). Then we can double back to each method to get them working one by one.

In [2]:
class MyLinearModel:
    def __init__(self):
        self.coefficients = None

    def fit(self, X, y):
        pass

    def predict(self, X_test):
        pass

**(2)** Now define your `fit` method. This is the method that should actually run your linear regression. In case you've forgotten your linear algebra, remember that for linear regressions, $\beta = (X'X)^{-1}X'Y$, a fact you can see explained in detail on page four [here](https://www.stat.purdue.edu/~boli/stat512/lectures/topic3.pdf).

Note that once you have written the code to do a linear regression, you'll need to put your outputs (your coefficents) somewhere. I recommend making an attribute for your class where you can store your coefficients. 

(As a reminder: the normal multiply operator (`*`) in `numpy` implies scalar multiplication. Use `@` for matrix multiplication). 

**HINT:** Remember that linear regressions require a vector of 1s in the `X` matrix. As the package writer, you get to decide whether users are expected to provide a matrix `X` that already has a vector of 1s, or whether you expect the user to provide a matrix `X` that doesn't have a vector of 1s (in which case you will need to add a vector of 1s before you fit the model).

In [3]:
import numpy as np


class MyLinearModel:
    def __init__(self):
        self.coefficients = None

    def fit(self, X, y):
        # add intercept term
        intercept = np.ones((X.shape[0], 1))
        X = np.hstack((intercept, X))

        # using the formula beta = (X'X)^-1 X'Y
        X_transpose = X.T
        self.coefficients = np.linalg.inv(X_transpose @ X) @ X_transpose @ y

    def predict(self, X_test):
        pass

**(3)** As you write code, it is good to test your code as you work. With that in mind, let's create some toy data. First, create a 100 x 2 matrix where each column is normally distributed. Then create a vector `y` that is a linear combination of those two columns **plus** a vector of normally distributed noise **and** a constant term. 

In other words, we want to create data where we *know* exactly what coefficients we should see so when we test our regression, we know if the results are accurate!

In [4]:
# random seed
np.random.seed(0)
X = np.random.normal(size=(100, 2))

# coefficients for linear combination (arbitrary choice)
true_coefficients = np.array([3, 5])
constant_term = 2

# Generate a vector y as a linear combination of X's columns
noise = np.random.normal(scale=1, size=100)
y = X @ true_coefficients + constant_term + noise

**(4)** Now test whether you `fit` method generates the correct coefficients. Remember the choice you made in Question 2 about whether your package expects the users' `X` matrix to include a vector of 1s when you test!

In [5]:
class MyLinearModel:
    def __init__(self):
        self.coefficients = None

    def fit(self, X, y):
        intercept = np.ones((X.shape[0], 1))
        X = np.hstack((intercept, X))
        X_transpose = X.T
        self.coefficients = np.linalg.inv(X_transpose @ X) @ X_transpose @ y

    def predict(self, X_test):
        pass


# create an instance of MyLinearModel
model = MyLinearModel()
model.fit(X, y)

# Retrieve the fitted coefficients
fitted_coefficients = model.coefficients

print("Fitted Coefficients:", fitted_coefficients)
print("Expected Coefficients: [2, 3, 5] (including the constant term)")

Fitted Coefficients: [1.94858887 3.1104649  4.9459629 ]
Expected Coefficients: [2, 3, 5] (including the constant term)


**(5)** Now let's make the statisticians proud, and in addition to storing the coefficients, let's store the standard errors for our estimated coefficients as another attribute. Recall that the simplest method of calculating the variance covariance matrix for $\beta$ is using the formula $\sigma^2 (X'X)^{-1}$, where $\sigma^2$ is the variance of the error terms of your regression. The standard errors for your coefficient estimates will be the diagonal values of that matrix. See page six [here](https://www.stat.purdue.edu/~boli/stat512/lectures/topic3.pdf) for a full derivation. 

In [6]:
class MyLinearModel:
    def __init__(self):
        self.coefficients = None
        self.std_errors = None

    def fit(self, X, y):
        intercept = np.ones((X.shape[0], 1))
        X = np.hstack((intercept, X))

        X_transpose = X.T
        self.coefficients = np.linalg.inv(X_transpose @ X) @ X_transpose @ y

        # Calculating the variance of the error terms
        y_pred = X @ self.coefficients
        residuals = y - y_pred
        error_variance = np.sum(residuals**2) / (X.shape[0] - X.shape[1])

        # Calculating the variance-covariance matrix of coefficients
        var_cov_matrix = error_variance * np.linalg.inv(X_transpose @ X)

        # Standard errors are the square roots of the diagonal elements
        self.std_errors = np.sqrt(np.diag(var_cov_matrix))

    def predict(self, X_test):
        pass


# test
model = MyLinearModel()
model.fit(X, y)

print("Coefficients:", model.coefficients)
print("Standard Errors:", model.std_errors)

Coefficients: [1.94858887 3.1104649  4.9459629 ]
Standard Errors: [0.09674321 0.09376987 0.09433688]


**(6)** Now let's also add an R-squared attribute to the model.

In [7]:
class MyLinearModel:
    def __init__(self):
        self.coefficients = None
        self.std_errors = None
        self.r_squared = None

    def fit(self, X, y):
        intercept = np.ones((X.shape[0], 1))
        X = np.hstack((intercept, X))

        X_transpose = X.T
        self.coefficients = np.linalg.inv(X_transpose @ X) @ X_transpose @ y

        y_pred = X @ self.coefficients
        residuals = y - y_pred

        error_variance = np.sum(residuals**2) / (X.shape[0] - X.shape[1])

        var_cov_matrix = error_variance * np.linalg.inv(X_transpose @ X)
        self.std_errors = np.sqrt(np.diag(var_cov_matrix))

        # R-squared
        total_variance = np.sum((y - y.mean()) ** 2)
        self.r_squared = 1 - (np.sum(residuals**2) / total_variance)

    def predict(self, X_test):
        pass


# test
model = MyLinearModel()
model.fit(X, y)

print("Coefficients:", model.coefficients)
print("Standard Errors:", model.std_errors)
print("R-squared:", model.r_squared)

Coefficients: [1.94858887 3.1104649  4.9459629 ]
Standard Errors: [0.09674321 0.09376987 0.09433688]
R-squared: 0.974901544340798


**(7)** Now we'll go ahead and cheat a little. Use `statsmodels` to fit your model with your toy data to ensure your standard errors and r-squared are correct!

In [12]:
import statsmodels.api as sm

In [15]:
X_with_intercept = sm.add_constant(X)
statsmodel = sm.OLS(y, X_with_intercept).fit()

print("Statsmodels Coefficients:", statsmodel.params)
print("Statsmodels Standard Errors:", statsmodel.bse)
print("Statsmodels R-squared:", statsmodel.rsquared)

Statsmodels Coefficients: [1.94858887 3.1104649  4.9459629 ]
Statsmodels Standard Errors: [0.09674321 0.09376987 0.09433688]
Statsmodels R-squared: 0.974901544340798


**(8)** Now implement `predict`! Then test it against your original `X` data -- do you get back something very close to your true `y`?

In [23]:
class MyLinearModel:
    def __init__(self):
        self.coefficients = None
        self.std_errors = None
        self.r_squared = None

    def fit(self, X, y):
        intercept = np.ones((X.shape[0], 1))
        X = np.hstack((intercept, X))

        X_transpose = X.T
        self.coefficients = np.linalg.inv(X_transpose @ X) @ X_transpose @ y

        y_pred = X @ self.coefficients
        residuals = y - y_pred

        error_variance = np.sum(residuals**2) / (X.shape[0] - X.shape[1])

        var_cov_matrix = error_variance * np.linalg.inv(X_transpose @ X)
        self.std_errors = np.sqrt(np.diag(var_cov_matrix))

        # R-squared
        total_variance = np.sum((y - y.mean()) ** 2)
        self.r_squared = 1 - (np.sum(residuals**2) / total_variance)

    def predict(self, X_test):
        intercept = np.ones((X_test.shape[0], 1))
        X_test = np.hstack((intercept, X_test))
        return X_test @ self.coefficients

In [24]:
np.random.seed(0)
X = np.random.rand(100, 2)
y = 2 * X[:, 0] + 3 * X[:, 1] + np.random.normal(0, 0.1, 100)

# Test your model
model = MyLinearModel()
model.fit(X, y)

# Predict using your model
y_pred = model.predict(X)

# Compare predicted values with true y
print("True y values:", y)
print("Predicted y values:", y_pred)

True y values: [3.3558587  2.73218315 2.67024507 3.50671142 3.02784683 3.36308804
 4.00782112 0.41221514 2.41575278 4.25078624 4.25469084 3.11006914
 2.27511489 3.15240759 2.37976835 2.88368506 2.70328557 1.82538353
 2.97156915 4.00111651 1.9497707  1.50698383 3.29989379 0.80929193
 1.68658962 2.31870295 2.21852027 0.67934175 2.1286146  1.50569256
 0.53862626 1.73242454 1.42538394 2.08759175 1.83489935 3.38557762
 3.76413045 1.47928119 0.97853127 0.93130892 1.9559344  2.28806177
 2.14569498 1.46197043 3.9028634  2.61543113 2.5225433  1.19391264
 1.29736166 1.51027088 2.16322438 4.28315059 2.25397075 2.89102454
 3.39542819 3.4651986  2.3699022  2.77047095 3.41158039 3.80006052
 2.95277966 3.88205304 2.89282063 0.93888513 2.0949854  2.48773878
 1.11943551 2.96069625 2.95417326 2.60478805 2.91142946 3.57071802
 3.66432212 2.93510944 4.28261753 2.85394279 2.11737841 2.83326966
 3.20632189 1.09998739 2.90493405 3.83625967 4.56107891 1.17105317
 1.91112645 1.16536    0.44227038 2.22939038 3.

**(9)** Finally, create the *option* of fitting the model with or without a constant term. In other words, create an option so that, if the user passes a numpy array *without* a constant term, your code will add a vector of 1s before fitting the model. As in `scikit-learn`, make this an option you set during initialization. 

In [26]:
class MyLinearModel:
    def __init__(self, fit_intercept=True):
        self.coefficients = None
        self.std_errors = None
        self.r_squared = None
        self.fit_intercept = fit_intercept

    def fit(self, X, y):
        if self.fit_intercept:
            X = sm.add_constant(X)

        X_transpose = X.T
        self.coefficients = np.linalg.inv(X_transpose @ X) @ X_transpose @ y

        y_pred = X @ self.coefficients
        residuals = y - y_pred

        error_variance = np.sum(residuals**2) / (X.shape[0] - X.shape[1])

        var_cov_matrix = error_variance * np.linalg.inv(X_transpose @ X)
        self.std_errors = np.sqrt(np.diag(var_cov_matrix))

        # R-squared
        total_variance = np.sum((y - y.mean()) ** 2)
        self.r_squared = 1 - (np.sum(residuals**2) / total_variance)

        return y_pred

    def predict(self, X_test):
        if self.fit_intercept:
            X_test = sm.add_constant(X_test)
        return X_test @ self.coefficients

In [27]:
model_with_intercept = MyLinearModel(fit_intercept=True)
y_pred_with_intercept = model_with_intercept.fit(X, y)

model_without_intercept = MyLinearModel(fit_intercept=False)
y_pred_without_intercept = model_without_intercept.fit(X, y)

# Compare predicted values with true y
print("True y values:", y)
print("Predicted y values (with intercept):", y_pred_with_intercept)
print("Predicted y values (without intercept):", y_pred_without_intercept)

True y values: [3.3558587  2.73218315 2.67024507 3.50671142 3.02784683 3.36308804
 4.00782112 0.41221514 2.41575278 4.25078624 4.25469084 3.11006914
 2.27511489 3.15240759 2.37976835 2.88368506 2.70328557 1.82538353
 2.97156915 4.00111651 1.9497707  1.50698383 3.29989379 0.80929193
 1.68658962 2.31870295 2.21852027 0.67934175 2.1286146  1.50569256
 0.53862626 1.73242454 1.42538394 2.08759175 1.83489935 3.38557762
 3.76413045 1.47928119 0.97853127 0.93130892 1.9559344  2.28806177
 2.14569498 1.46197043 3.9028634  2.61543113 2.5225433  1.19391264
 1.29736166 1.51027088 2.16322438 4.28315059 2.25397075 2.89102454
 3.39542819 3.4651986  2.3699022  2.77047095 3.41158039 3.80006052
 2.95277966 3.88205304 2.89282063 0.93888513 2.0949854  2.48773878
 1.11943551 2.96069625 2.95417326 2.60478805 2.91142946 3.57071802
 3.66432212 2.93510944 4.28261753 2.85394279 2.11737841 2.83326966
 3.20632189 1.09998739 2.90493405 3.83625967 4.56107891 1.17105317
 1.91112645 1.16536    0.44227038 2.22939038 3.